In [1]:
# start
# 18:00
# 18:39
    
    
import random
import numpy as np
import math

In [10]:
def genDat(n,l):
    rotL = lambda x: x[1:] + [x[0]]
    
    rand = lambda v: map(lambda z: random.choice([0,1]), [0]*v)
    x  = map(rand, [l] * n)
    y  = map(rotL, x)
    
    x = map(np.array, x)
    y = map(np.array, y)
    
    return zip(x,y)   

def logsigmoid(z):
    return 1.0/(1 + math.exp(-z))

def dlogsigmoid(z):
    return logsigmoid(z)*(1 - logsigmoid(z))

In [24]:
class NN:
    def __init__(_,ll,lr=0.05):
        _.ll = ll
        _.lr = lr
        _.fA  = np.vectorize(logsigmoid)
        _.dfA = np.vectorize(dlogsigmoid)
        _.W, _.B = _.genWB()
        
    def genWB(_):
        W = [np.random.randn(n,m) for (n,m) in zip(_.ll[:-1], _.ll[1:])]
        B = [np.random.randn(m)   for m     in _.ll[1:]]
        
        return W,B
    
    def ff(_,x):
        V,VF = [None]*len(_.ll), [None]*len(_.ll)
        V[0] = VF[0] = x
        
        for i in range(len(_.W)):
            V[i+1]  = np.dot(np.transpose(_.W[i]), VF[i]) + _.B[i]
            VF[i+1] = _.fA(V[i+1])
        
        return V,VF
    
    def ff_pure(_,x):        
        v = x
        
        for i in range(len(_.W)):
            v  = _.fA(np.dot(np.transpose(_.W[i]), v) + _.B[i])
                    
        return v
    
    
    def fb(_, V, VF, y):
        G  = [None] * len(_.W)
        GW = [None] * len(_.W)
        
        G[-1] = (VF[-1] - y)*_.dfA(V[-1])
        
        for i in range(len(_.W)-2, -1, -1):            
            G[i] = np.multiply(np.dot(_.W[i+1], G[i+1]), _.dfA(V[i+1]))
            
        for i in range(len(_.W)-1, -1, -1):            
            GW[i] = np.transpose( np.outer(G[i], VF[i]) )
            
        return GW, G
    
    def trainStep(_,x,y):
        V,VF  = _.ff(x)
        GW,GB = _.fb(V,VF,y)
        
        _.W = _.W - np.multiply(GW, _.lr)
        _.B = _.B - np.multiply(GB, _.lr)
        
        out = _.ff_pure(x)
        
        [e1,e2] = _.err(VF[-1], y), _.err(out, y)
        return [e1, e2]
    
    @classmethod
    def err(cls, out,y):
        return math.sqrt(sum((out - y)** 2))
    
    def train(_, data):
        return [_.trainStep(x,y) for (x,y) in data]
    
    def test(_, data):
        for (x,y) in data:
            out = _.ff_pure(x)
            err = _.err(out, y)
            print "x = {}, y = {}, out = {}, e = {}".format(x,y,out,err)
            
layers = [[3,5,3], [3,4,3]]


dat = genDat(100000,3)

for ll in layers:
    print "ll = {}".format(ll)
    nn = NN(ll)
    nn.train(dat)
    nn.test(dat[:5])

ll = [3, 5, 3]
x = [0 0 0], y = [0 0 0], out = [ 0.02428733  0.02490493  0.02725379], e = 0.0441916188129
x = [1 0 1], y = [0 1 1], out = [ 0.02436384  0.97754815  0.99067265], e = 0.0344192014833
x = [1 1 0], y = [1 0 1], out = [ 0.98476079  0.03079752  0.98245797], e = 0.0385803501124
x = [1 1 0], y = [1 0 1], out = [ 0.98476079  0.03079752  0.98245797], e = 0.0385803501124
x = [1 1 1], y = [1 1 1], out = [ 0.97203345  0.97586964  0.97480224], e = 0.0447138613573
ll = [3, 4, 3]
x = [0 0 0], y = [0 0 0], out = [ 0.02478079  0.01810799  0.01314677], e = 0.0333889835737
x = [1 0 1], y = [0 1 1], out = [ 0.02324147  0.96968475  0.97528376], e = 0.0454980509889
x = [1 1 0], y = [1 0 1], out = [ 0.98086338  0.02012407  0.96911163], e = 0.0415364842635
x = [1 1 0], y = [1 0 1], out = [ 0.98086338  0.02012407  0.96911163], e = 0.0415364842635
x = [1 1 1], y = [1 1 1], out = [ 0.97212614  0.98313059  0.98734398], e = 0.0349528796192
